In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)


from datetime import date

today = str(date.today())
end_date = today

import glob

path = r'data/final/'  # use your path
allFiles = glob.glob(path + "/*.csv")

dftrain = pd.DataFrame()

for _file in allFiles[:-2]:
        try:
            t = pd.read_csv(_file,header=0)
            if len(t[(t["fvalue"].isnull())]) > 0:
                print("Some nulls in " + _file)
            dftrain = pd.concat([dftrain,t])
        except:
            pass
print("Last file loaded: " + _file)
test_file = allFiles[-2:-1][0]
print("Test file " + test_file)
dfTest = pd.read_csv(test_file,header=0)

predict_file = 'data/final/2017-01-26.csv'
print("Predict file " + predict_file)
dfPredict = pd.read_csv(predict_file, header=0)

out_file = predict_file.replace("final","final2")
print("Out file " + out_file)

Last file loaded: data/final/2017-01-24.csv
Test file data/final/2017-01-25.csv
Predict file data/final/2017-01-26.csv
Out file data/final2/2017-01-26.csv


In [3]:
scaler = StandardScaler()
features = ['Salary','AvgPointsPerGame'
    ,'7GameAvg','FloorAvg'
    ,'4GameAvg','value','fvalue'
    ,'atHome','LastGame','O/U','odds',"dvp"]

In [5]:
df_train = dftrain
X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train["Final"].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [6]:
df_test = dfTest

X_test = scaler.transform(df_test[features].values)
y_test = df_test["Final"].values

In [7]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

In [8]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=130,
                              random_state=1,
                              verbose=False,
                              warm_start=True)
modelRFR = modelRFR.fit(X_train, y_train)

#dfPredict = scaler.transform(X_test[features].values)
X_test = scaler.transform(df_test[features].values)
y_tree_prediction = modelRFR.predict(X_test)
print ("\naccuracy", np.sum(abs(y_tree_prediction-y_test)<5) / float(len(y_test)))


accuracy 0.871428571429


In [8]:
#y_tree_prediction
#pd.concat([dfTest["Name"],pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(32, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
model.compile(optimizer='adam', metrics=['accuracy'],loss="mae")

model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=150, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 177.0226399898529 seconds


In [24]:
scores = model.evaluate(X_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/280 [==>...........................] - ETA: 0sacc: 35.71%


In [25]:
#dfpredict = pd.read_csv('data/output/3017-01-21.csv', header=0)

#X_test = scaler.transform(dfPredict1[features].values)

y_predict = model.predict(X_test)
#pd.DataFrame(y_predict).describe()
# (y_predict - y_test) < 10
print(np.sum(abs(y_predict.T[0]-y_test) < 5)/float(len(y_test)))
#y_predict[[range(0,len(y_predict))]]
#print ("\naccuracy", np.sum(abs(y_predict-y_test)<10) / float(len(y_test)))
#y_tree_predict = modelRFR.predict(X_test)

0.878571428571


In [22]:
output = pd.concat([dfTest,
            pd.DataFrame(y_predict,columns=["NN"]),
            pd.DataFrame(y_tree_prediction,columns=["RF"])]
          ,axis=1)
#output.to_csv('data/output/2017-01-24.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [56]:
output.loc[:,("Name","NN","RF","AvgPointsPerGame","Final","O/U","odds","dvp")].sort_values(by=["O/U","NN","dvp"], ascending=False).head()

,Name,NN,RF,AvgPointsPerGame,Final,O/U,odds,dvp
7,Isaiah Thomas,55.215801,56.937500,43.813,60.25,229.0,4.0,9.5
1,James Harden,53.166351,47.100543,60.651,58.50,229.0,-4.0,19.0
30,Al Horford,43.038368,45.725543,36.140,47.75,229.0,4.0,22.0
49,Jae Crowder,41.257439,42.086957,26.403,45.00,229.0,4.0,25.0
166,Nene Hilario,30.168289,32.241848,15.976,0.00,229.0,-4.0,21.0


In [55]:
##  Predictions for Today!!!

In [66]:
scaler1 = StandardScaler()
features = ['Salary','AvgPointsPerGame'
    ,'7GameAvg','FloorAvg'
    ,'4GameAvg','value','fvalue'
    ,'atHome','LastGame','O/U','odds',"dvp"]

X_predict = scaler1.fit_transform(dfPredict[features].values)
y_tree_prediction = modelRFR.predict(X_predict) ## Random Forest
y_predict = model.predict(X_predict) ## Neural Networks prediction

In [69]:
output = pd.concat([dfPredict,
            pd.DataFrame(y_predict,columns=["NN"]),
            pd.DataFrame(y_tree_prediction,columns=["RF"])]
          ,axis=1)

In [72]:
output.head()
print(out_file)

data/final2/2017-01-26.csv


In [73]:
output.to_csv(out_file, sep=',', encoding='utf-8', index=False, float_format='%.3f')